In [6]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [7]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [8]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [9]:
# bj_hm_score 冰鉴-火眸分
# my_zl_score2 蚁盾分2.0
# bj_qy_score 冰鉴-青云分
# grade 评分等级
# br_ra_score 百融-融安分
# score_201048 融360
# score_202098 融360
# score_202100 融360
# td212_score 极信-td212
# my_zl_score 蚁盾分1.0
# jx_yl_UPPE082 近12个月成功扣款频度占比
# jx_yl_UPPD029 近1个月资金不足交易频度
# jx_fico_score  极信-fico
# jx_yl_UPPC196 近1个月消费频度
# xy_qj_A22160004 申请命中消⾦类机构数

In [27]:
df = pd.read_excel('F:/租后表分析数据_ld/京东/data_jxz.xlsx')
df.loc[:, 'riskLevel'] = df.jd_user_level_risk.apply(lambda x: jsonpath(json.loads(x), '$..riskLevel')[0])
df.loc[:, '拒绝理由_new'] = df.拒绝理由.apply(lambda x: x.strip('[]'))
df = df[df.拒绝理由_new!='']
df.loc[:, '火眸分'] = np.where((df.bj_hm_score<500)&(df.bj_hm_score>0), 1, 0)
df.loc[:, '信用等级低于D'] = np.where(df.riskLevel.isin(['E', 'F']), 1, 0)
df.loc[:, '蚁盾分2.0'] = np.where(df.my_zl_score2>=65, 1, 0)
df.loc[:, '青云分'] = np.where((df.bj_qy_score<500)&(df.bj_qy_score>0), 1, 0)
df.loc[:, '评分等级低于D'] = np.where(df.grade.isin(['E', 'F']), 1, 0)
df.loc[:, '融360'] = np.where((df.score_201048<=424)|(df.score_202098<=417)|((df.score_202100<=400)&(df.br_ra_score<740)), 1, 0)
df.loc[:, '极信_212'] = np.where(df.td212_score>95, 1, 0)
df.loc[:, 'strategy_240829'] = np.where(((df['my_zl_score'] > 34.384)&((df['jx_yl_UPPE082'] <= 0.015)|(df['jx_yl_UPPE082'].isnull())))|
                            ((df['jx_yl_UPPD029'] > 8.0 )&(df['jx_fico_score'] <= 550.0))|
                            ((df['jx_yl_UPPC196'] <= 5.5)&(df['xy_qj_A22160004'] > 0.5)), 1, 0)
df[['order_number', '拒绝理由', '拒绝理由_new', '火眸分', '信用等级低于D', '蚁盾分2.0', '青云分', '评分等级低于D', '融360', '极信_212', 'strategy_240829']].to_excel('F:/京东.xlsx', index=False)
# df.loc[:, 'rejected'] = np.where((df.火眸分==1)|(df.信用等级低于D==1)|(df.蚁盾分2==1)|(df.青云分==1)|(df.评分等级低于D==1)|(df.融360==1)|(df.极信==1)|(df.strategy_240829==1), df.columns, '')